In [1]:
from neomodel import config, db
import os

# Configure the connection to the Neo4j database
config.DATABASE_URL = os.getenv("NEO4J_URI")

# check connection
try:
    # Attempt to run a simple query
    db.cypher_query("MATCH (n) RETURN count(n)")
    print("Connection successful!")
except Exception as e:
    print(f"Connection failed: {e}")

Connection successful!


In [2]:
import json
from typing import Any, Dict  
from neomodel import StructuredNode, StringProperty, RelationshipTo, StructuredRel  
from pydantic import BaseModel  
  
  
# Define relationship models  
class ReferencesRel(StructuredRel):  
    name = StringProperty()  
    description = StringProperty()  
  
  
# Define node models  
class Interface(StructuredNode):  
    name = StringProperty(unique_index=True)  
    summary = StringProperty()  
    namespace = StringProperty()  
    assembly = StringProperty()  
    references = RelationshipTo("Interface", "REFERENCES", model=ReferencesRel)  
  
  
class Metadata(BaseModel):  
    name: str  
    summary: str  
    namespace: str  
    assembly: str  
    type_references: list[str]  
  
  
def extract_metadata(key: str, value: Any) -> Metadata:  
    return Metadata(  
        name=key,  
        summary=value["summary"],  
        namespace=value["namespace"],  
        assembly=value["assembly"],  
        type_references=value["type_references"],  
    )  
  
  
def create_graph_from_interfaces(interfaces: Dict[str, Any]):  
    interface_nodes : Dict[str, Interface] = {}  
  
    # Create interface nodes  
    for key, value in interfaces.items():  
        metadata = extract_metadata(key, value)  
        if metadata.name not in interface_nodes:  
            interface_node = Interface(  
                name=metadata.name,  
                summary=metadata.summary,  
                namespace=metadata.namespace,  
                assembly=metadata.assembly,  
            ).save()  
            interface_nodes[metadata.name] = interface_node  
  
    # Create references between interface nodes  
    for key, value in interfaces.items():  
        metadata = extract_metadata(key, value)  
        source_node = interface_nodes[metadata.name]  
        for ref_name in metadata.type_references:  
            if ref_name in interface_nodes and ref_name != metadata.name:  # Skip self-references  
                target_node = interface_nodes[ref_name]  
                if not source_node.references.is_connected(target_node):  
                    source_node.references.connect(target_node) 


# Load all interfaces from the single JSON file
with open("scraped_domain_knowledge.json", "r", encoding="utf8") as f:  
    interfaces: Dict[str, Any] = json.load(f)  
  
create_graph_from_interfaces(interfaces)  
